In [ ]:
import pandas as pd 
%matplotlib inline
import numpy as np
import statsmodels.api as sm
import sys
stdin, stdout, stderr = sys.stdin, sys.stdout, sys.stderr
sys.stdin, sys.stdout, sys.stderr = stdin, stdout, stderr
import pandas as pd 
import pylab, math
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
from datetime import time, datetime
import datetime, time
import datetime as dt
import warnings
from sklearn import metrics
from IPython.core.pylabtools import figsize
from scipy.stats import ttest_ind
from scipy import stats
from scipy.stats.mstats import winsorize
from numpy.random import permutation
import seaborn as sns
from matplotlib.ticker import FixedFormatter

# Dictionary of datasets
| Column        | Meaning                            |
|---------------|------------------------------------|
| imp_id        | impression ID                      |
| unit_id       | ad ID                              |
| unit_tag      | treatment condition of ad          |
| exp_tag       | treatment condition of impression  |
| p_date        | date                               |
| p_hourmin     | hour                               |
| pxtr          | predicted ctr*cvr                  |
| retention_cnt | # of retention days                |
| cvt_cnt       | # of conversions during cold start |
| click         | # of clicks                        |
| imp           | # of impressions                   |
| auto_cpa_bid  | real-time bid                      |
| target_bid    | bid set by advertisers             |
| cost_total    | real cost of ads                   |
| target_cost   | expected cost of ads               |

# Section 1: Figure 1 Ad retention

## Figure 1 with observational data

Dataset [retention_data.csv'] contains observational data before the experiment. Each row records the basic information ('imp', 'click_cnt', 'cvr_cnt', 'target_cost', 'cost_total', 'target_bid'), aggregated at day level (p_date) of a specific ad (unit_id).
Note: the code and data of matching and IV methods in paper Appendix are not given here.

In [ ]:
df = pd.read_csv('retention_data.csv')
df = df[['unit_id', 'imp', 'click_cnt', 'cvr_cnt', 'target_cost',
                                   'cost_total', 'target_bid', 'p_date']]
df=df.dropna()
max_cvr = 100
ad_full_pct = np.zeros(max_cvr)
ad_cvr_cnt = np.zeros(max_cvr)

for unit_id, g in df.groupby("unit_id"):
    g_np = g.values
    cvr_cnt = g_np[0,3]
    for i in range(min(int(cvr_cnt+1), max_cvr)):
        ad_cvr_cnt[i]+=1
    if g.shape[0]>=14:#remain in more than 2 weeks
        for i in range(min(int(cvr_cnt+1), max_cvr)):
            ad_full_pct[i]+=1
ad_full_pct = ad_full_pct/ad_cvr_cnt

#linearly scaled
figsize(6, 4)
fig, ax = plt.subplots(1, 1)
plt.rcParams['savefig.dpi'] = 1200
plt.rcParams['figure.dpi'] = 1200
ax.yaxis.set_ticks_position('both')
ax.xaxis.set_ticks_position('both')
plt.rcParams['xtick.direction'] = 'in'
plt.rcParams['ytick.direction'] = 'in'
plt.rc('font',family='Times New Roman')
error_interval=1.26*np.sqrt(ad_full_pct*(1-ad_full_pct)/ad_cvr_cnt)
min_value=min(ad_full_pct-error_interval)
max_value=max(ad_full_pct+error_interval)
plt.plot(range(max_cvr), (ad_full_pct-min_value)/(max_value-min_value), color='steelblue',linestyle='--',label='Average Retention Rate')
plt.fill_between(range(max_cvr), (ad_full_pct-error_interval-min_value)/(max_value-min_value), 
	(ad_full_pct+error_interval-min_value)/(max_value-min_value), color='lightskyblue', alpha=.3,label='95% Confidence Interval')
plt.xlabel('Average Conversions per Day during the Cold-Start Period')
plt.ylabel('Retention Rate of New Ads')
plt.legend(loc="lower right") 
ax.set_ylim(0,1)
ax.set_xlim(0,100)
plt.savefig('cold_start_reward.png', dpi=1200,bbox_inches='tight',pad_inches=0.0) 

# Empirical analysis in Section 5 

## Section 5 Randomization check of field experiments

Dataset [randomization_check_data_block.csv] is used for randomization check and constains ad performance data before the experiments. Each row records the basic information ('target_cost','cost_total','cvt_cnt','auto_cpa_bid','target_bid','imp','click'), aggregated at day level ('p_date') of a specific ad ('unit_id') with the field ('unit_tag') indicating whether the ad is in the treatment group or not.

In [ ]:
df=pd.read_csv('randomization_check_data_block.csv')
#column includes: 'unit_id','p_date','unit_tag','target_cost','cost_total','cvt_cnt','auto_cpa_bid','target_bid','imp','click'

#check cold start reward and success rate
df_exp_cvt=df[(df['unit_tag']=='exp')].groupby('unit_id').sum().reset_index()
df_base_cvt=df[(df['unit_tag']=='base')].groupby('unit_id').sum().reset_index()

df_exp_cvt=df_exp_cvt[df_exp_cvt['imp']>0][['unit_id','cvt_cnt']]
df_base_cvt=df_base_cvt[df_base_cvt['imp']>0][['unit_id','cvt_cnt']]

df_exp_bid=df[(df['unit_tag']=='exp')].groupby('unit_id')['target_bid'].mean().reset_index()
df_base_bid=df[(df['unit_tag']=='base')].groupby('unit_id')['target_bid'].mean().reset_index()

df_exp = pd.merge(df_exp_cvt, df_exp_bid, on='unit_id', how='left')
df_base = pd.merge(df_base_cvt, df_base_bid, on='unit_id', how='left')

df_exp['cold_start_rate']= df_exp['cvt_cnt'].apply(lambda x: 1 if x>=10 else 0)
df_base['cold_start_rate']= df_base['cvt_cnt'].apply(lambda x: 1 if x>=10 else 0)

df_exp['cold_start_reward']=np.minimum(df_exp['cvt_cnt'], 10)*df_exp['target_bid']*2/1000.0
df_base['cold_start_reward']=np.minimum(df_base['cvt_cnt'], 10)*df_base['target_bid']*2/1000.0
print('check cold start reward')
print(ttest_ind(df_exp['cold_start_reward'],df_base['cold_start_reward']))
print('exp', np.mean(df_exp['cold_start_reward']), np.std(df_exp['cold_start_reward']))
print('base',np.mean(df_base['cold_start_reward']), np.std(df_base['cold_start_reward']))
print('--------------------')
print('check cold start success rate')
print(ttest_ind(df_exp['cold_start_rate'],df_base['cold_start_rate']))
print('exp',np.mean(df_exp['cold_start_rate']), np.std(df_exp['cold_start_rate']))
print('base',np.mean(df_base['cold_start_rate']), np.std(df_base['cold_start_rate']))
print('--------------------')


#check # of impressions
print('check # of impressions')
df_exp=df[(df['unit_tag']=='exp')].groupby('unit_id').sum().reset_index()
df_base=df[(df['unit_tag']=='base')].groupby('unit_id').sum().reset_index()
df_exp=df_exp[df_exp['imp']>0]
df_base=df_base[df_base['imp']>0]
df_exp=df_exp['imp']
df_base=df_base['imp']
print('mean:',np.mean(df_exp),np.mean(df_base))
print('std:',np.std(df_exp),np.std(df_base))
print(ttest_ind(df_exp, df_base))
print('--------------------')


#check # of conversions
print('check # of conversions')
df_exp=df[(df['unit_tag']=='exp')].groupby('unit_id').sum().reset_index()
df_base=df[(df['unit_tag']=='base')].groupby('unit_id').sum().reset_index()
df_exp=df_exp[df_exp['imp']>0]
df_base=df_base[df_base['imp']>0]
df_exp=df_exp[df_exp['cvt_cnt']<1000]
df_base=df_base[df_base['cvt_cnt']<1000]
df_exp=df_exp['cvt_cnt']
df_base=df_base['cvt_cnt']
print('mean:',np.mean(df_exp),np.mean(df_base))
print('std:',np.std(df_exp),np.std(df_base))
print(ttest_ind(df_exp, df_base))
print('--------------------')


#check total revenue
print('check total revenue')
df_exp=df[(df['unit_tag']=='exp')].groupby('unit_id').sum().reset_index()
df_base=df[(df['unit_tag']=='base')].groupby('unit_id').sum().reset_index()
df_exp=df_exp[df_exp['imp']>0]
df_base=df_base[df_base['imp']>0]
df_exp=df_exp['cost_total']
df_base=df_base['cost_total']
print('mean:',np.mean(df_exp),np.mean(df_base))
print('std:',np.std(df_exp),np.std(df_base))
print(ttest_ind(df_exp, df_base))
print('--------------------')


#check # of ads
print('check # of ads')
df_exp=df[(df['unit_tag']=='exp')].groupby('unit_id').sum().reset_index()
df_base=df[(df['unit_tag']=='base')].groupby('unit_id').sum().reset_index()
df_exp=df_exp[df_exp['imp']>0]
df_base=df_base[df_base['imp']>0]
df_exp=df_exp['click']
df_base=df_base['click']
print('mean:',np.mean(df_exp),np.mean(df_base))
print('std:',np.std(df_exp),np.std(df_base))
print(ttest_ind(df_exp, df_base))
print('--------------------')


#check CTR CVR
df_exp=df[(df['unit_tag']=='exp')].groupby('unit_id').sum().reset_index()
df_base=df[(df['unit_tag']=='base')].groupby('unit_id').sum().reset_index()
df_exp=df_exp[df_exp['imp']>0]
df_base=df_base[df_base['imp']>0]
df_exp=df_exp[df_exp['click']>0]
df_base=df_base[df_base['click']>0]

df_exp['ctr']=df_exp['click']/df_exp['imp']
df_exp['cvr']=df_exp['cvt_cnt']/df_exp['click']

df_base['ctr']=df_base['click']/df_base['imp']
df_base['cvr']=df_base['cvt_cnt']/df_base['click']

df_exp=df_exp[df_exp['ctr']<1]
df_base=df_base[df_base['ctr']<1]
df_exp=df_exp[df_exp['cvr']<1]
df_base=df_base[df_base['cvr']<1]

print('CTR check:')
print('mean:',np.mean(df_exp['ctr']),np.mean(df_base['ctr']))
print('std:',np.std(df_exp['ctr']),np.std(df_base['ctr']))
print(ttest_ind(df_exp['ctr'], df_base['ctr']))
print('--------------------')

print('CVR check:')
print('mean:',np.mean(df_exp['cvr']),np.mean(df_base['cvr']))
print('std:',np.std(df_exp['cvr']),np.std(df_base['cvr']))
print(ttest_ind(df_exp['cvr'], df_base['cvr']))
print('--------------------')





# Empirical analysis in Section 6 

## Section 6.1 Short-Term Performance of Our oSBL Algorithm

Dataset [exp_short_term_results.csv] contains the ad performance data during the experiment. Each row records the basic performance information aggregated at hour level ('p_date','p_hourmin') of a specific ad ('unit_id') with assigned conditions ('exp_tag','unit_tag').

Dataset [exp_cost_results.csv] contains the revenue performance aggregated at hour level during the experiment.

In [ ]:
df = pd.read_csv('exp_short_term_results.csv')#aggregate at ad*day_hour level with different experiment conditions
df=df[['unit_id','p_date','p_hourmin','exp_tag','unit_tag','target_cost','cost_total','cvt_cnt','auto_cpa_bid','target_bid','imp','click']]

#Panel A: Effects on the Cold Start at the Ad Level
print('===================================================')
print('Panel A')
#impressions
df_exp=df[(df['exp_tag']=='exp1') & (df['unit_tag']=='exp')].groupby('unit_id').sum().reset_index()
df_base=df[(df['exp_tag']=='base1') & (df['unit_tag']=='ctrl')].groupby('unit_id').sum().reset_index()
df_exp=df_exp[df_exp['imp']>0]
df_base=df_base[df_base['imp']>0]
df_exp=df_exp['imp']
df_base=df_base['imp']
print('Check impressions:')
print('mean of exp and base groups:',np.mean(df_exp), np.mean(df_base))
print('std:',np.std(df_exp),np.std(df_base))
print('ttest', ttest_ind(df_exp, df_base))
print('----------------------------------------------------')

#clicks
df_exp=df[(df['exp_tag']=='exp1') & (df['unit_tag']=='exp')].groupby('unit_id').sum().reset_index()
df_base=df[(df['exp_tag']=='base1') & (df['unit_tag']=='ctrl')].groupby('unit_id').sum().reset_index()
df_exp=df_exp[df_exp['imp']>0]
df_base=df_base[df_base['imp']>0]
df_exp=df_exp['click']
df_base=df_base['click']
print('Check clicks:')
print('mean of exp and base groups:',np.mean(df_exp),np.mean(df_base))
print('std:',np.std(df_exp),np.std(df_base))
print('ttest', ttest_ind(df_exp, df_base))
print('----------------------------------------------------')


#conversions
print('Check conversions:')
df_exp=df[(df['exp_tag']=='exp1') & (df['unit_tag']=='exp')].groupby('unit_id').sum().reset_index()
df_base=df[(df['exp_tag']=='base1') & (df['unit_tag']=='ctrl')].groupby('unit_id').sum().reset_index()
df_exp=df_exp[df_exp['imp']>0]
df_base=df_base[df_base['imp']>0]
df_exp=df_exp['cvt_cnt']
df_base=df_base['cvt_cnt']
print('mean of exp and base groups:',np.mean(df_exp),np.mean(df_base))
print('std:',np.std(df_exp),np.std(df_base))
print('ttest', ttest_ind(df_exp, df_base))
print('----------------------------------------------------')


#bid price
df_exp=df[(df['exp_tag']=='exp1') & (df['unit_tag']=='exp')].groupby('unit_id').mean().reset_index()
df_base=df[(df['exp_tag']=='base1') & (df['unit_tag']=='ctrl')].groupby('unit_id').mean().reset_index()
df_exp=df_exp[df_exp['imp']>0]
df_base=df_base[df_base['imp']>0]
df_exp=df_exp['target_bid']
df_base=df_base['target_bid']
print('Check bid prices:')
print('mean of exp and base groups:',np.mean(df_exp),np.mean(df_base))
print('std:',np.std(df_exp),np.std(df_base))
print('ttest', ttest_ind(df_exp, df_base))
print('----------------------------------------------------')


#cold start success rate
target=10
bound=2000000
B11_cvt = df[(df['exp_tag']=='exp1') & (df['unit_tag']=='exp')].groupby('unit_id').sum().reset_index()
B22_cvt = df[(df['exp_tag']=='base1') & (df['unit_tag']=='ctrl')].groupby('unit_id').sum().reset_index()
B11_cvt=B11_cvt[B11_cvt['imp']>0]
B22_cvt=B22_cvt[B22_cvt['imp']>0]
B11_cvt=B11_cvt['cvt_cnt']
B22_cvt=B22_cvt['cvt_cnt']
B11=0
B22=0
for i in list(B11_cvt):
    if i>=target and i<=bound:
        B11+=1
for i in list(B22_cvt):
    if i>=target and i<=bound:
        B22+=1
B11=B11/B11_cvt.count()
B22=B22/B22_cvt.count()
print('Check cold start success rate')
print('ttest', ttest_ind(B11_cvt, B22_cvt))
print('----------------------------------------------------')

#Panel B: Effects of the Algorithm on Short-Term Revenue and the Objective Value
print('===================================================')
print('Panel B: Effects of the Algorithm on Short-Term Revenue and the Objective Value')
df_cost = pd.read_csv('exp_cost_results.csv')
df_cost=df_cost.sort_values(by=['p_date','p_hourmin'])
dates=[20200523,20200524,20200525,20200526,20200527,20200528,20200529]
x= [datetime.datetime.strptime(str(d), '%Y%m%d').date() for d in dates]
plt.gcf().autofmt_xdate()
fig=plt.figure(figsize=(15,5))
ax1=fig.add_subplot(1,1,1) 
date1_1 = dt.datetime(2020,5, 23, 0)
date1_2 = dt.datetime(2020, 5, 29, 20)
delta1 = dt.timedelta(hours=1)
dates1 = matplotlib.dates.drange(date1_1, date1_2, delta1)
y1 = np.random.rand(len(dates1))
dateFmt = matplotlib.dates.DateFormatter('%Y-%m-%d')
ax1.xaxis.set_major_formatter(dateFmt)
daysLoc = matplotlib.dates.DayLocator()
hoursLoc = matplotlib.dates.HourLocator(interval=6)
ax1.xaxis.set_major_locator(daysLoc)
ax1.xaxis.set_minor_locator(hoursLoc)
ax1.plot(dates1[0:164],df_cost[df_cost['exp_tag']=='base1']['cost_total'].reset_index()['cost_total'], label='base', color='b',linestyle='-.')
ax1.plot(dates1[0:164],df_cost[df_cost['exp_tag']=='exp1']['cost_total'].reset_index()['cost_total'],label='exp', color='r',linestyle=':')
plt.xlabel('hour')
plt.ylabel('cost')
plt.title('Total revenue by groups')
plt.legend()
plt.show()
df_exp_cvt=df[(df['exp_tag']=='exp1') & (df['unit_tag']=='exp')].groupby('unit_id').sum().reset_index()
df_base_cvt=df[(df['exp_tag']=='base1') & (df['unit_tag']=='ctrl')].groupby('unit_id').sum().reset_index()
df_exp_cvt=df_exp_cvt[df_exp_cvt['imp']>0][['unit_id','cvt_cnt','imp','click']]
df_base_cvt=df_base_cvt[df_base_cvt['imp']>0][['unit_id','cvt_cnt','imp','click']]
df_exp_bid=df[(df['exp_tag']=='exp1') & (df['unit_tag']=='exp')].groupby('unit_id')['target_bid'].mean().reset_index()
df_base_bid=df[(df['exp_tag']=='base1') & (df['unit_tag']=='ctrl')].groupby('unit_id')['target_bid'].mean().reset_index()
df_exp = pd.merge(df_exp_cvt, df_exp_bid, on='unit_id', how='left')
df_base = pd.merge(df_base_cvt, df_base_bid, on='unit_id', how='left')

df_exp['cold_start_rate']= df_exp['cvt_cnt'].apply(lambda x: 1 if x>=10 else 0)
df_base['cold_start_rate']= df_base['cvt_cnt'].apply(lambda x: 1 if x>=10 else 0)

df_exp['cold_start_reward']=np.minimum(df_exp['cvt_cnt'], 10)*df_exp['target_bid']*2/1000.0
df_base['cold_start_reward']=np.minimum(df_base['cvt_cnt'], 10)*df_base['target_bid']*2/1000.0

print(df_exp['cold_start_reward'].sum(), df_exp.groupby('unit_id').unit_id.nunique().count())
print(df_base['cold_start_reward'].sum(), df_base.groupby('unit_id').unit_id.nunique().count())
print('------------revenue--------------')
print('Note: current code applies t-test at hour granularity, our paper report t-test at impression granularity')
rev_base = df_cost[df_cost['exp_tag']=='base1']['cost_total'].reset_index()['cost_total'].to_numpy()
rev_exp = df_cost[df_cost['exp_tag']=='exp1']['cost_total'].reset_index()['cost_total'].to_numpy()
print('ttest:', ttest_ind(rev_exp, rev_base))
print('exp', np.mean(df_exp['cold_start_reward']), np.std(df_exp['cold_start_reward']))
print('base',np.mean(df_base['cold_start_reward']), np.std(df_base['cold_start_reward']))
print('------------cold start reward--------------')
print('ttest:', ttest_ind(df_exp['cold_start_reward'],df_base['cold_start_reward']))
print('exp', np.mean(df_exp['cold_start_reward']), np.std(df_exp['cold_start_reward']))
print('base',np.mean(df_base['cold_start_reward']), np.std(df_base['cold_start_reward']))
print('------------cold start rate--------------')
print('ttest:', ttest_ind(df_exp['cold_start_rate'],df_base['cold_start_rate']))
print('exp',np.mean(df_exp['cold_start_rate']), np.std(df_exp['cold_start_rate']))
print('base',np.mean(df_base['cold_start_rate']), np.std(df_base['cold_start_rate']))

##Panel C: Effects of the Algorithm on Advertiser Costs
print('===================================================')
print('Panel C: Effects of the Algorithm on Advertiser Costs')
level_1_exp=[]
level_1_base=[]
level_2_exp=[]
level_2_base=[]
level_3_exp=[]
level_3_base=[]
df_tempt=df.dropna(axis=0,how='any')
for time_, g in df_tempt.groupby(['p_date']):
    exp_cvt_cost = g[(g['exp_tag']=='exp1') & (g['unit_tag']=='exp')].groupby('unit_id').sum()
    ctrl_cvt_cost = g[(g['exp_tag']=='base1') & (g['unit_tag']=='ctrl')].groupby('unit_id').sum()
    exp_cvt_cost = exp_cvt_cost[(exp_cvt_cost['cost_total']>0) & (exp_cvt_cost['target_cost']>0)]
    ctrl_cvt_cost = ctrl_cvt_cost[(ctrl_cvt_cost['cost_total']>0) & (ctrl_cvt_cost['target_cost']>0)]
    exp_cvt_cost['cost_ratio'] = exp_cvt_cost['cost_total']/exp_cvt_cost['target_cost']
    ctrl_cvt_cost['cost_ratio'] = ctrl_cvt_cost['cost_total']/ctrl_cvt_cost['target_cost']
    #print('+-30%')
    level_1_exp.append(exp_cvt_cost[(exp_cvt_cost['cost_ratio']<1.3) & (exp_cvt_cost['cost_ratio']>0.7)]['cost_ratio'].count()/exp_cvt_cost['cost_ratio'].count())
    level_1_base.append(ctrl_cvt_cost[(ctrl_cvt_cost['cost_ratio']<1.3) & (ctrl_cvt_cost['cost_ratio']>0.7)]['cost_ratio'].count()/ctrl_cvt_cost['cost_ratio'].count())
    
    #30%~100%
    #print('30%~100%')
    level_2_exp.append(exp_cvt_cost[(exp_cvt_cost['cost_ratio']>1.3) & (exp_cvt_cost['cost_ratio']<2)]['cost_ratio'].count()/exp_cvt_cost['cost_ratio'].count())
    level_2_base.append(ctrl_cvt_cost[(ctrl_cvt_cost['cost_ratio']>1.3) & (ctrl_cvt_cost['cost_ratio']<2)]['cost_ratio'].count()/ctrl_cvt_cost['cost_ratio'].count())
    
    #>100%
    #print('>100%')
    level_3_exp.append(exp_cvt_cost[(exp_cvt_cost['cost_ratio']>2)]['cost_ratio'].count()/exp_cvt_cost['cost_ratio'].count())
    level_3_base.append(ctrl_cvt_cost[(ctrl_cvt_cost['cost_ratio']>2)]['cost_ratio'].count()/ctrl_cvt_cost['cost_ratio'].count())
level_1_exp = np.array(level_1_exp)
level_1_base = np.array(level_1_base)
level_2_exp = np.array(level_1_exp)
level_2_base = np.array(level_1_base)
level_3_exp = np.array(level_1_exp)
level_3_base = np.array(level_1_base)
print('ttest at +-30%:', ttest_ind(level_1_exp,level_1_base))
print('ttest at 30%~100%:', ttest_ind(level_2_exp,level_2_base))
print('ttest at >100%:', ttest_ind(level_3_exp,level_3_base))
print('===================================================')



## Section 6.2 Long-term effects of oSBL

Dataset [oSBL_ad_level_long_term_results.csv] contains post-experiment data for around 3 months. Each row records the basic performance information aggregated at day level ('p_date') of a specific ad ('unit_id') with assigned conditions ('unit_tag').

In [ ]:
#Figure 6 
#this code cannot run on the given sample dataset [oSBL_ad_level_long_term_results.csv] because 
#the sample dataset does not cover all post-experiment periods from 5/30/2020 to 8/21/2020. 
#Thus a dimension mismatch bug in line 29-30.

df_sbl_long = pd.read_csv('oSBL_ad_level_long_term_results.csv')#aggregate at day*ad level
df_sbl_long = df_sbl_long[['p_date','unit_id','unit_tag','target_cost','cost_total','cvt_cnt','auto_cpa_bid','target_bid','imp','click']]
df_sbl_long['unit_tag']= df_sbl_long['unit_tag'].apply(lambda x: 'exp' if x=='exp' else 'ctrl')

df_sbl_long_2 = df_sbl_long.copy()
figsize(6, 4)
plt.rcParams['savefig.dpi'] = 300 
plt.rcParams['figure.dpi'] = 300 
df_sbl_long_cnt=df_sbl_long_2.groupby(['p_date','unit_tag']).unit_id.nunique().reset_index().sort_values(by=['p_date'])
plt.gcf().autofmt_xdate() 
fig=plt.figure(figsize=(15,5))
ax1=fig.add_subplot(1,1,1) 
date1_1 = dt.datetime(2020,5, 30, 0)
date1_2 = dt.datetime(2020,8, 21, 0)
delta1 = dt.timedelta(days=1)
dates1 = matplotlib.dates.drange(date1_1, date1_2, delta1)
y1 = np.random.rand(len(dates1))
dateFmt = matplotlib.dates.DateFormatter('%Y-%m-%d')
ax1.xaxis.set_major_formatter(dateFmt)
 
daysLoc = matplotlib.dates.DayLocator(interval=7)
ax1.xaxis.set_major_locator(daysLoc)

ax1.plot(dates1[0:83],df_sbl_long_cnt[df_sbl_long_cnt['unit_tag']=='ctrl']['unit_id'], label='Control', color='b',linestyle='-.')
ax1.plot(dates1[0:83],df_sbl_long_cnt[df_sbl_long_cnt['unit_tag']=='exp']['unit_id'], label='Treatment', color='r',linestyle=':')

plt.xlabel('date')
plt.ylabel('Market Thickness of Experimental Ads')
plt.ylim([0,0.6])
plt.legend()
plt.show()
fig.savefig('market_thickness.png', dpi=300)

In [ ]:
df_sbl_long = pd.read_csv('oSBL_ad_level_long_term_results.csv')#aggregate at day*ad level
df_sbl_long = df_sbl_long[['p_date','unit_id','unit_tag','target_cost','cost_total','cvt_cnt','auto_cpa_bid','target_bid','imp','click']]
df_sbl_long['unit_tag']= df_sbl_long['unit_tag'].apply(lambda x: 'exp' if x=='exp' else 'ctrl')
df_sbl_long_2 = df_sbl_long.copy()
df_test = df_sbl_long_2.groupby(['unit_id','unit_tag']).sum().reset_index()

rescale_ = 1 #true rescale number is not revealed in the code
limits = [0, 0.9]#winsorization limit
df_test['imp']=rescale_*df_test['imp']
df_test['cost_total']=rescale_*df_test['cost_total']
df_test['target_cost']=rescale_*df_test['target_cost']

print('Post-exp lifetime impressions')
print(stats.ttest_ind(df_test[df_test['unit_tag']=='exp']['imp'],
                df_test[df_test['unit_tag']=='ctrl']['imp']))
print('exp=',df_test[df_test['unit_tag']=='exp']['imp'].mean())
print('ctrl=',df_test[df_test['unit_tag']=='ctrl']['imp'].mean())
print('effect= ', df_test[df_test['unit_tag']=='exp']['imp'].mean()/df_test[df_test['unit_tag']=='ctrl']['imp'].mean()-1)

print('std exp=',df_test[df_test['unit_tag']=='exp']['imp'].std())
print('std ctrl=',df_test[df_test['unit_tag']=='ctrl']['imp'].std())
print('==================================================')

print('Post-exp lifetime log-impressions')
print(stats.ttest_ind(np.log(df_test[df_test['unit_tag']=='exp']['imp']),
                np.log(df_test[df_test['unit_tag']=='ctrl']['imp'])))
print('exp=',np.log(df_test[df_test['unit_tag']=='exp']['imp']).mean())
print('ctrl=',np.log(df_test[df_test['unit_tag']=='ctrl']['imp']).mean())
print('effect= ',np.log(df_test[df_test['unit_tag']=='exp']['imp']).mean()/np.log(df_test[df_test['unit_tag']=='ctrl']['imp']).mean()-1)

print('std exp=',np.log(df_test[df_test['unit_tag']=='exp']['imp']).std())
print('std ctrl=',np.log(df_test[df_test['unit_tag']=='ctrl']['imp']).std())


print('==================================================')

print('Post-exp lifetime winsorize-impressions')
print(stats.ttest_ind(winsorize(df_test[df_test['unit_tag']=='exp']['imp'], limits),
                winsorize(df_test[df_test['unit_tag']=='ctrl']['imp'], limits)))
print('exp=',winsorize(df_test[df_test['unit_tag']=='exp']['imp'], limits).mean())
print('ctrl=',winsorize(df_test[df_test['unit_tag']=='ctrl']['imp'],limits).mean())
print('effect= ',winsorize(df_test[df_test['unit_tag']=='exp']['imp'], limits).mean()/winsorize(df_test[df_test['unit_tag']=='ctrl']['imp'],limits).mean()-1)

print('std exp=',winsorize(df_test[df_test['unit_tag']=='exp']['imp'],limits).std())
print('std ctrl=',winsorize(df_test[df_test['unit_tag']=='ctrl']['imp'],limits).std())
print('==================================================')

df_test['cost_total']=df_test['cost_total']+1
df_test['target_cost']=df_test['target_cost']+1
print('Post-exp lifetime cost_total')
print(stats.ttest_ind(df_test[df_test['unit_tag']=='exp']['cost_total'],
                df_test[df_test['unit_tag']=='ctrl']['cost_total']))
print('exp=',df_test[df_test['unit_tag']=='exp']['cost_total'].mean())
print('ctrl=',df_test[df_test['unit_tag']=='ctrl']['cost_total'].mean())
print('effect= ', df_test[df_test['unit_tag']=='exp']['cost_total'].mean()/df_test[df_test['unit_tag']=='ctrl']['cost_total'].mean()-1)
print('std exp=',df_test[df_test['unit_tag']=='exp']['cost_total'].std())
print('std ctrl=',df_test[df_test['unit_tag']=='ctrl']['cost_total'].std())
print('==================================================')



print('Post-exp lifetime log cost_total')
print(stats.ttest_ind(np.log(df_test[df_test['unit_tag']=='exp']['cost_total']),
                np.log(df_test[df_test['unit_tag']=='ctrl']['cost_total'])))
print('exp=',np.log(df_test[df_test['unit_tag']=='exp']['cost_total']).mean())
print('ctrl=',np.log(df_test[df_test['unit_tag']=='ctrl']['cost_total']).mean())
print('effect= ',np.log(df_test[df_test['unit_tag']=='exp']['cost_total']).mean()/np.log(df_test[df_test['unit_tag']=='ctrl']['cost_total']).mean()-1)

print('std exp=',np.log(df_test[df_test['unit_tag']=='exp']['cost_total']).std())
print('std ctrl=',np.log(df_test[df_test['unit_tag']=='ctrl']['cost_total']).std())
print('==================================================')

print('Post-exp lifetime lifetime winsorize cost_total')
print(stats.ttest_ind(winsorize(df_test[df_test['unit_tag']=='exp']['cost_total'], limits),
                winsorize(df_test[df_test['unit_tag']=='ctrl']['cost_total'], limits)))
print('exp=',winsorize(df_test[df_test['unit_tag']=='exp']['cost_total'], limits).mean())
print('ctrl=',winsorize(df_test[df_test['unit_tag']=='ctrl']['cost_total'],limits).mean())
print('effect= ',winsorize(df_test[df_test['unit_tag']=='exp']['cost_total'], limits).mean()/winsorize(df_test[df_test['unit_tag']=='ctrl']['cost_total'],limits).mean()-1)

print('std exp=',winsorize(df_test[df_test['unit_tag']=='exp']['cost_total'],limits).std())
print('std ctrl=',winsorize(df_test[df_test['unit_tag']=='ctrl']['cost_total'],limits).std())
print('==================================================')

print('Post-exp lifetime target_cost')
print(stats.ttest_ind(df_test[df_test['unit_tag']=='exp']['target_cost'],
                df_test[df_test['unit_tag']=='ctrl']['target_cost']))
print('exp=',df_test[df_test['unit_tag']=='exp']['target_cost'].mean())
print('ctrl=',df_test[df_test['unit_tag']=='ctrl']['target_cost'].mean())
print('std exp=',df_test[df_test['unit_tag']=='exp']['target_cost'].std())
print('std ctrl=',df_test[df_test['unit_tag']=='ctrl']['target_cost'].std())
print('==================================================')

print('Post-exp lifetime log target_cost')
print(stats.ttest_ind(np.log(df_test[df_test['unit_tag']=='exp']['target_cost']),
                np.log(df_test[df_test['unit_tag']=='ctrl']['target_cost'])))
print('exp=',np.log(df_test[df_test['unit_tag']=='exp']['target_cost']).mean())
print('ctrl=',np.log(df_test[df_test['unit_tag']=='ctrl']['target_cost']).mean())
print('std exp=',np.log(df_test[df_test['unit_tag']=='exp']['target_cost']).std())
print('std ctrl=',np.log(df_test[df_test['unit_tag']=='ctrl']['target_cost']).std())
print('==================================================')

print('Post-exp lifetime winsorize target_cost')
print(stats.ttest_ind(winsorize(df_test[df_test['unit_tag']=='exp']['target_cost'], limits),
                winsorize(df_test[df_test['unit_tag']=='ctrl']['target_cost'], limits)))
print('exp=',winsorize(df_test[df_test['unit_tag']=='exp']['target_cost'], limits).mean())
print('ctrl=',winsorize(df_test[df_test['unit_tag']=='ctrl']['target_cost'],limits).mean())
print('std exp=',winsorize(df_test[df_test['unit_tag']=='exp']['target_cost'],limits).std())
print('std ctrl=',winsorize(df_test[df_test['unit_tag']=='ctrl']['target_cost'],limits).std())
print('==================================================')

df_test['imp']=df_test['imp']+1
df_test['ctr']=df_test['click']/df_test['imp']
df_test['ctr']=rescale_*df_test['ctr']


print('Post-exp average CTR')
print(stats.ttest_ind(df_test[df_test['unit_tag']=='exp']['ctr'],
                df_test[df_test['unit_tag']=='ctrl']['ctr']))
print('exp=',df_test[df_test['unit_tag']=='exp']['ctr'].mean())
print('ctrl=',df_test[df_test['unit_tag']=='ctrl']['ctr'].mean())
print('effect= ', df_test[df_test['unit_tag']=='exp']['ctr'].mean()/df_test[df_test['unit_tag']=='ctrl']['ctr'].mean()-1)

print('std exp=',df_test[df_test['unit_tag']=='exp']['ctr'].std())
print('std ctrl=',df_test[df_test['unit_tag']=='ctrl']['ctr'].std())
print('==================================================')

df_test['click']=df_test['click']+1
df_test['cvr']=df_test['cvt_cnt']/df_test['click']
df_test=df_test[df_test['cvr']<1]
df_test['cvr']=rescale_*df_test['cvr']

print('Post-exp average CVR')
print(stats.ttest_ind(df_test[df_test['unit_tag']=='exp']['cvr'],
                df_test[df_test['unit_tag']=='ctrl']['cvr']))
print('exp=',df_test[df_test['unit_tag']=='exp']['cvr'].mean())
print('ctrl=',df_test[df_test['unit_tag']=='ctrl']['cvr'].mean())
print('std exp=',df_test[df_test['unit_tag']=='exp']['cvr'].std())
print('std ctrl=',df_test[df_test['unit_tag']=='ctrl']['cvr'].std())
print('==================================================')


df_test = df_sbl_long_2.groupby(['unit_id','unit_tag']).mean().reset_index()
df_test['target_bid']=rescale_*df_test['target_bid']
df_test['auto_cpa_bid']=rescale_*df_test['auto_cpa_bid']

print('Post-exp average target_bid')
print(stats.ttest_ind(df_test[df_test['unit_tag']=='exp']['target_bid'],
                df_test[df_test['unit_tag']=='ctrl']['target_bid']))
print('exp=',df_test[df_test['unit_tag']=='exp']['target_bid'].mean())
print('ctrl=',df_test[df_test['unit_tag']=='ctrl']['target_bid'].mean())
print('std exp=',df_test[df_test['unit_tag']=='exp']['target_bid'].std())
print('std ctrl=',df_test[df_test['unit_tag']=='ctrl']['target_bid'].std())
print('==================================================')

print('Post-exp average auto_cpa_bid')
print(stats.ttest_ind(df_test[df_test['unit_tag']=='exp']['auto_cpa_bid'],
                df_test[df_test['unit_tag']=='ctrl']['auto_cpa_bid']))
print('exp=',df_test[df_test['unit_tag']=='exp']['auto_cpa_bid'].mean())
print('ctrl=',df_test[df_test['unit_tag']=='ctrl']['auto_cpa_bid'].mean())
print('effect= ', df_test[df_test['unit_tag']=='exp']['auto_cpa_bid'].mean()/df_test[df_test['unit_tag']=='ctrl']['auto_cpa_bid'].mean()-1)

print('std exp=',df_test[df_test['unit_tag']=='exp']['auto_cpa_bid'].std())
print('std ctrl=',df_test[df_test['unit_tag']=='ctrl']['auto_cpa_bid'].std())
print('==================================================')





## Section 6.3 Global Treatment Effect of Our oSBL Algorithm on Advertising Revenue

Each row in data [auction_level_data.csv] is one record of the auction: for a specific auction ('imp_id') happened at date ('p_date'), one ad ('unit_id') join the auction with key information including predicted CTRxCVR ('pxtr'), bid price ('auto_cpa_bid') and new ad indicator ('is_new').

In [ ]:
df_auction_long = pd.read_csv('auction_level_data.csv')
df_auction_long = df_auction_long[['imp_id','p_date','unit_id','pxtr','auto_cpa_bid','is_new']]
df_auction_long['cpm'] =  df_auction_long['pxtr']*df_auction_long['auto_cpa_bid']
df_new_bonus = pd.DataFrame(columns=['p_date', 'unit_id'])
for i, g in df_auction_long.groupby('p_date'):
    g_ = g.sort_values(by=['cpm'], ascending=False, kind='mergesort').drop_duplicates('imp_id')
    g_ = g_[g_['is_new']==1].groupby(['p_date','unit_id'])['pxtr'].sum().reset_index()
    g_ = g_.sort_values(by=['pxtr'], ascending=False, kind='mergesort')[['p_date', 'unit_id']]
    num_new_ads = g[g['is_new']==1].unit_id.nunique()
    df_new_bonus = pd.concat([df_new_bonus, g_.head(int(num_new_ads*0.3))])#give 30% of new ads shadow bids
df_new_bonus['is_bonus'] = 0
df_auction_long = pd.merge(df_auction_long, df_new_bonus, on=['p_date', 'unit_id'], how='left')
df_auction_long = df_auction_long.fillna(1)
df_auction_long.loc[df_auction_long['is_new']==0, 'is_bonus']= 0
df_auction_long['new_old']=0#if new ads turn into old das
df_auction_long.loc[(df_auction_long['is_new']==0),'new_old'] = 1
df_auction_long = df_auction_long[['imp_id','p_date','unit_id','pxtr','auto_cpa_bid','is_new','cpm','is_bonus', 'new_old']]

revenue_increase = np.zeros([10, 11,11])
i_cnt = 0
j_cnt = 0
for kk in range(10):
    for ctr_increase in np.linspace(0,0.2,11):
        for thickness in np.linspace(0,0.05,11):
            df_auction_long_s = df_auction_long.copy()
            df_auction_long_s['new_old']=0
            df_auction_long_s.loc[(df_auction_long['is_new']==0),'new_old'] = 1
            #simulation with ctr increas and market increase
            df_auction_long_s['new_pxtr'] = df_auction_long_s['pxtr']*(1+ctr_increase*(df_auction_long_s['new_old']))


            df_auction_long_s['cpm'] = (1+2*df_auction_long_s['is_bonus'])*df_auction_long_s['new_pxtr']*(df_auction_long_s['auto_cpa_bid'])
            df_auction_long_s['real_cpm'] = df_auction_long_s['new_pxtr']*(df_auction_long_s['auto_cpa_bid'])


            df_extra = df_auction_long_s[df_auction_long_s['new_old']==1].sample(frac=thickness)
            from numpy.random import permutation
            idx = permutation(len(df_extra))
            x = np.array(df_extra[['cpm','real_cpm']])[idx, :]
            df_extra.loc[:,'cpm'] = x[:, 0]
            df_extra.loc[:,'real_cpm'] = x[:, 1]



            df = pd.concat([df_auction_long_s, df_extra]).sort_values(by=['cpm'], ascending=False, kind='mergesort').drop_duplicates('imp_id').groupby(['p_date','is_new']).sum()['real_cpm']
            df_auction_long_s['real_cpm'] = df_auction_long_s['pxtr']*(df_auction_long_s['auto_cpa_bid'])
            df_no_sbl = df_auction_long_s.sort_values(by=['real_cpm'], ascending=False, kind='mergesort').drop_duplicates('imp_id').groupby(['p_date','is_new']).sum()['real_cpm']
            df = df.reset_index()
            df_no_sbl = df_no_sbl.reset_index()


            a=np.array(df_no_sbl[df_no_sbl['is_new']==0]['real_cpm'])+np.array(df_no_sbl[df_no_sbl['is_new']==1]['real_cpm'])
            b=np.array(df[df['is_new']==0]['real_cpm'])+np.array(df[df['is_new']==1]['real_cpm'])


            revenue_increase[kk][i_cnt][j_cnt] = np.sum(b)/np.sum(a)-1
            j_cnt+=1
        j_cnt=0
        i_cnt+=1
    i_cnt=0
    #np.save('simulation_long_term_0', revenue_increase)

y = np.linspace(0,0.2,11)#ctr
x = np.linspace(0,0.05,11)#thickness
Z = np.zeros([11,11]) 
res = pd.DataFrame(columns=('Relative Thickness Increase', 'Relative CTR Increase' , 'revenue'))

X, Y = np.meshgrid(x, y)
for i in range(11):
    for j in range(11):
        Z[i, j] = np.mean(revenue_increase[:,i,j])
        res=res.append({'Relative Thickness Increase': X[i, j], 'Relative CTR Increase': Y[i, j], 'revenue': Z[i, j]}, ignore_index=True)
    
res = res.pivot("Relative Thickness Increase", "Relative CTR Increase", "revenue").round(4)
fig, ax = plt.subplots(1, 1)
ax = sns.heatmap(res, annot=True)
ax.invert_yaxis()
#fig.savefig('Simulation.png', dpi=300) 


#simulations under [0.01, 0.02, 0.03] retention rate increase
#with varying CTR increase and beta coefficients
revenue_increase_2 = np.zeros([10, 11,11])#0.01
revenue_increase_3 = np.zeros([10, 11,11])#0.02
revenue_increase_4 = np.zeros([10, 11,11])#0.03
i_cnt = 0
j_cnt = 0
for kk in range(10):
    
    for ctr_increase in np.linspace(0,0.15,11):

        for beta_ in np.linspace(0,3,11):
            df_auction_long_s = df_auction_long.copy()

            df_auction_long_s['new_old']=0
            df_auction_long_s.loc[(df_auction_long['is_new']==0),'new_old'] = 1
            df_auction_long_s['new_pxtr'] = df_auction_long_s['pxtr']*(1+ctr_increase*(df_auction_long_s['new_old']))


            df_auction_long_s['cpm'] = (1+beta_*df_auction_long_s['is_bonus'])*df_auction_long_s['new_pxtr']*(df_auction_long_s['auto_cpa_bid'])
            df_auction_long_s['real_cpm'] = df_auction_long_s['new_pxtr']*(df_auction_long_s['auto_cpa_bid'])


            df_extra = df_auction_long_s[df_auction_long_s['new_old']==1].sample(frac=0.01)
            
            idx = permutation(len(df_extra))
            x = np.array(df_extra[['cpm','real_cpm']])[idx, :]
            df_extra.loc[:,'cpm'] = x[:, 0]
            df_extra.loc[:,'real_cpm'] = x[:, 1]



            df = pd.concat([df_auction_long_s, df_extra]).sort_values(by=['cpm'], ascending=False, kind='mergesort').drop_duplicates('imp_id').groupby(['p_date','is_new']).sum()['real_cpm']
            df_auction_long_s['real_cpm'] = df_auction_long_s['pxtr']*(df_auction_long_s['auto_cpa_bid'])
            df_no_sbl = df_auction_long_s.sort_values(by=['real_cpm'], ascending=False, kind='mergesort').drop_duplicates('imp_id').groupby(['p_date','is_new']).sum()['real_cpm']
            df = df.reset_index()
            df_no_sbl = df_no_sbl.reset_index()


            a=np.array(df_no_sbl[df_no_sbl['is_new']==0]['real_cpm'])+np.array(df_no_sbl[df_no_sbl['is_new']==1]['real_cpm'])
            b=np.array(df[df['is_new']==0]['real_cpm'])+np.array(df[df['is_new']==1]['real_cpm'])


            revenue_increase_2[kk][i_cnt][j_cnt] = np.sum(b)/np.sum(a)-1
            j_cnt+=1
        j_cnt=0
        i_cnt+=1
    i_cnt=0
    #np.save('simulation_long_term_2', revenue_increase_2)
    
    
i_cnt = 0
j_cnt = 0
for kk in range(10):
    
    for ctr_increase in np.linspace(0,0.15,11):

        for beta_ in np.linspace(0,3,11):
            df_auction_long_s = df_auction_long.copy()
            df_auction_long_s['new_old']=0
            df_auction_long_s.loc[(df_auction_long['is_new']==0),'new_old'] = 1
            df_auction_long_s['new_pxtr'] = df_auction_long_s['pxtr']*(1+ctr_increase*(df_auction_long_s['new_old']))

            df_auction_long_s['cpm'] = (1+beta_*df_auction_long_s['is_bonus'])*df_auction_long_s['new_pxtr']*(df_auction_long_s['auto_cpa_bid'])
            df_auction_long_s['real_cpm'] = df_auction_long_s['new_pxtr']*(df_auction_long_s['auto_cpa_bid'])


            df_extra = df_auction_long_s[df_auction_long_s['new_old']==1].sample(frac=0.02)
            idx = permutation(len(df_extra))
            x = np.array(df_extra[['cpm','real_cpm']])[idx, :]
            df_extra.loc[:,'cpm'] = x[:, 0]
            df_extra.loc[:,'real_cpm'] = x[:, 1]

            df = pd.concat([df_auction_long_s, df_extra]).sort_values(by=['cpm'], ascending=False, kind='mergesort').drop_duplicates('imp_id').groupby(['p_date','is_new']).sum()['real_cpm']
            df_auction_long_s['real_cpm'] = df_auction_long_s['pxtr']*(df_auction_long_s['auto_cpa_bid'])
            df_no_sbl = df_auction_long_s.sort_values(by=['real_cpm'], ascending=False, kind='mergesort').drop_duplicates('imp_id').groupby(['p_date','is_new']).sum()['real_cpm']
            df = df.reset_index()
            df_no_sbl = df_no_sbl.reset_index()


            a=np.array(df_no_sbl[df_no_sbl['is_new']==0]['real_cpm'])+np.array(df_no_sbl[df_no_sbl['is_new']==1]['real_cpm'])
            b=np.array(df[df['is_new']==0]['real_cpm'])+np.array(df[df['is_new']==1]['real_cpm'])


            revenue_increase_3[kk][i_cnt][j_cnt] = np.sum(b)/np.sum(a)-1
            j_cnt+=1
        j_cnt=0
        i_cnt+=1
    i_cnt=0

    #np.save('simulation_long_term_3', revenue_increase_3)
    
    
i_cnt = 0
j_cnt = 0
for kk in range(10):
    
    for ctr_increase in np.linspace(0,0.15,11):

        for beta_ in np.linspace(0,3,11):
            df_auction_long_s = df_auction_long.copy()

            df_auction_long_s['new_old']=0
            df_auction_long_s.loc[(df_auction_long['is_new']==0),'new_old'] = 1
            df_auction_long_s['new_pxtr'] = df_auction_long_s['pxtr']*(1+ctr_increase*(df_auction_long_s['new_old']))


            df_auction_long_s['cpm'] = (1+beta_*df_auction_long_s['is_bonus'])*df_auction_long_s['new_pxtr']*(df_auction_long_s['auto_cpa_bid'])
            df_auction_long_s['real_cpm'] = df_auction_long_s['new_pxtr']*(df_auction_long_s['auto_cpa_bid'])


            df_extra = df_auction_long_s[df_auction_long_s['new_old']==1].sample(frac=0.03)
            #from numpy.random import permutation
            idx = permutation(len(df_extra))
            x = np.array(df_extra[['cpm','real_cpm']])[idx, :]
            df_extra.loc[:,'cpm'] = x[:, 0]
            df_extra.loc[:,'real_cpm'] = x[:, 1]



            df = pd.concat([df_auction_long_s, df_extra]).sort_values(by=['cpm'], ascending=False, kind='mergesort').drop_duplicates('imp_id').groupby(['p_date','is_new']).sum()['real_cpm']
            df_auction_long_s['real_cpm'] = df_auction_long_s['pxtr']*(df_auction_long_s['auto_cpa_bid'])
            df_no_sbl = df_auction_long_s.sort_values(by=['real_cpm'], ascending=False, kind='mergesort').drop_duplicates('imp_id').groupby(['p_date','is_new']).sum()['real_cpm']
            df = df.reset_index()
            df_no_sbl = df_no_sbl.reset_index()


            a=np.array(df_no_sbl[df_no_sbl['is_new']==0]['real_cpm'])+np.array(df_no_sbl[df_no_sbl['is_new']==1]['real_cpm'])
            b=np.array(df[df['is_new']==0]['real_cpm'])+np.array(df[df['is_new']==1]['real_cpm'])


            revenue_increase_4[kk][i_cnt][j_cnt] = np.sum(b)/np.sum(a)-1
            j_cnt+=1
        j_cnt=0
        i_cnt+=1
    i_cnt=0
    #np.save('simulation_long_term_4', revenue_increase_4)